<a href="https://colab.research.google.com/github/slowvak/AI-Deep-Learning-Lab/blob/master/COVID_CXR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import logging
import os


from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report
import numpy as np
from PIL import Image

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import CrossEntropyLoss

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import models
from torchvision import transforms

log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


In [0]:


log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def load_model_weights(model, state_dict, verbose=True):
    """
    Loads the model weights from the state dictionary. Function will only load
    the weights which have matching key names and dimensions in the state
    dictionary.

    :param state_dict: Pytorch model state dictionary
    :param verbose: bool, If True, the function will print the
        weight keys of parametares that can and cannot be loaded from the
        checkpoint state dictionary.
    :return: The model with loaded weights
    """
    new_state_dict = model.state_dict()
    non_loadable, loadable = set(), set()

    for k, v in state_dict.items():
        if k not in new_state_dict:
            non_loadable.add(k)
            continue

        if v.shape != new_state_dict[k].shape:
            non_loadable.add(k)
            continue

        new_state_dict[k] = v
        loadable.add(k)

    if verbose:
        log.info("### Checkpoint weights that WILL be loaded: ###")
        {log.info(k) for k in loadable}

        log.info("### Checkpoint weights that CANNOT be loaded: ###")
        {log.info(k) for k in non_loadable}

    model.load_state_dict(new_state_dict)
    return model


def to_device(tensor, gpu=False):
    """
    Places a Pytorch Tensor object on a GPU or CPU device.

    :param tensor: Pytorch Tensor object
    :param gpu: bool, Flag which specifies GPU placement
    :return: Tensor object
    """
    return tensor.cuda() if gpu else tensor.cpu()


def clf_metrics(predictions, targets, average='macro'):
    f1 = f1_score(targets, predictions, average=average)
    precision = precision_score(targets, predictions, average=average)
    recall = recall_score(targets, predictions, average=average)
    acc = accuracy_score(targets, predictions)

    return acc, f1, precision, recall


def get_learning_rate(optimizer):
    """
    Retrieves the current learning rate. If the optimizer doesn't have
    trainable variables, it will raise an error.
    :param optimizer: Optimizer object
    :return: float, Current learning rate
    """
    if len(optimizer.param_groups) > 0:
        return optimizer.param_groups[0]['lr']
    else:
        raise ValueError('No trainable parameters.')


In [0]:



class COVIDxFolder(Dataset):
    def __init__(self, img_dir, labels_file, transforms):
        self.img_pths, self.labels = self._prepare_data(img_dir, labels_file)
        self.transforms = transforms

    def _prepare_data(self, img_dir, labels_file):
        with open(labels_file, 'r') as f:
            labels_raw = f.readlines()

        labels, img_pths = [], []
        for i in range(len(labels_raw)):
            data = labels_raw[i].split()
            img_name = data[1]
            img_pth = os.path.join(img_dir, img_name)
            img_pths.append(img_pth)
            labels.append(mapping[data[2]])

        return img_pths, labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = Image.open(self.img_pths[idx]).convert("RGB")
        img_tensor = self.transforms(img)

        label = self.labels[idx]
        label_tensor = torch.tensor(label, dtype=torch.long)

        return img_tensor, label_tensor


In [0]:

def train_transforms(width, height):
    trans_list = [
        transforms.Resize((height, width)),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([
            transforms.RandomAffine(degrees=20,
                                    translate=(0.15, 0.15),
                                    scale=(0.8, 1.2),
                                    shear=5)], p=0.5),
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.3, contrast=0.3)], p=0.5),
        transforms.ToTensor()
    ]
    return transforms.Compose(trans_list)


def val_transforms(width, height):
    trans_list = [
        transforms.Resize((height, width)),
        transforms.ToTensor()
    ]
    return transforms.Compose(trans_list)


In [0]:

class Trainable(nn.Module):
    """
    Wraps an arbitrary module with a Trainable module. The Trainable module
    is used as a wrapper for freezing and thawing module layers.
    """
    def __init__(self, module, name, trainable=True):
        super().__init__()
        self.module = module
        self.name = name
        self.trainable_switch(trainable)

    def __call__(self, *args, **kwargs):
        return self.module(*args, **kwargs)

    def trainable_switch(self, trainable):
        """
        Makes module layers trainable or not.

        :param trainable: bool, False to freeze the layers, True to unfreeze
         them.
        """
        for p in self.parameters():
            p.requires_grad = trainable


def ConvBn2d(in_dim, out_dim, kernel_size,
             activation=nn.LeakyReLU(0.1, inplace=True)):
    """
    Wraps Conv2D, Batch Normalization 2D, and an arbitrary activation layers
     with a nn.Sequential layer.

    :param in_dim: int, Input feature map dimension
    :param out_dim: int, Output feature map dimension
    :param kernel_size: int or tuple, Convolution kernel size
    :return: nn.Sequential structure containing above listed network layers
    """
    padding = kernel_size // 2
    net = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=kernel_size,
                  padding=padding, bias=False),
        nn.BatchNorm2d(out_dim),
        activation)
    return net



In [0]:

class COVIDNext50(nn.Module):
    def __init__(self, n_classes):
        super(COVIDNext50, self).__init__()
        self.n_classes = n_classes
        trainable = True

        # Layers
        backbone = models.resnext50_32x4d(pretrained=True)
        self.block0 = Trainable(nn.Sequential(
                                    backbone.conv1,
                                    backbone.bn1,
                                    backbone.relu,
                                    backbone.maxpool),
                                trainable=trainable,
                                name="conv1")
        self.block1 = Trainable(backbone.layer1,
                                trainable=trainable,
                                name="block1")
        self.block2 = Trainable(backbone.layer2,
                                trainable=trainable,
                                name="block2")
        self.block3 = Trainable(backbone.layer3,
                                trainable=trainable,
                                name="block3")
        self.block4 = Trainable(backbone.layer4,
                                trainable=trainable,
                                name="block4")
        self.backbone_end = Trainable(nn.Sequential(
                                        ConvBn2d(2048, 512, 3),
                                        ConvBn2d(512, 1024, 1),
                                        ConvBn2d(1024, 512, 3)),
                                      name="back",
                                      trainable=True)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.logits = Trainable(nn.Linear(512, n_classes),
                                name="logits",
                                trainable=True)

    def forward(self, input):
        net = input
        for layer in [self.block0, self.block1, self.block2, self.block3,
                      self.block4]:
            net = layer(net)
        net = self.backbone_end(net)
        net = self.avg_pool(net)
        net = torch.squeeze(net)
        return self.logits(net)

    def probability(self, logits):
        return nn.functional.softmax(logits, dim=-1)


In [0]:

def save_model(model, config):
    if isinstance(model, torch.nn.DataParallel):
        # Save without the DataParallel module
        model_dict = model.module.state_dict()
    else:
        model_dict = model.state_dict()

    state = {
        "state_dict": model_dict,
        "global_step": config['global_step'],
        "clf_report": config['clf_report']
    }
    f1_macro = config['clf_report']['macro avg']['f1-score'] * 100
    name = "{}_F1_{:.2f}_step_{}.pth".format(config['name'],
                                             f1_macro,
                                             config['global_step'])
    model_path = os.path.join(config['save_dir'], name)
    torch.save(state, model_path)
    log.info("Saved model to {}".format(model_path))



In [0]:

def validate(data_loader, model, best_score, global_step):
    model.eval()
    gts, predictions = [], []

    log.info("Validation started...")
    for data in data_loader:
        imgs, labels = data
        imgs = to_device(imgs, gpu=gpu)

        with torch.no_grad():
            logits = model(imgs)
            #probs = probability(logits)
            probs = nn.functional.softmax(logits, dim=-1)
            preds = torch.argmax(probs, dim=1).cpu().numpy()

        labels = labels.cpu().detach().numpy()

        predictions.extend(preds)
        gts.extend(labels)

    predictions = np.array(predictions, dtype=np.int32)
    gts = np.array(gts, dtype=np.int32)
    acc, f1, prec, rec = clf_metrics(predictions=predictions,
                                          targets=gts,
                                          average="macro")
    report = classification_report(gts, predictions, output_dict=True)

    log.info("VALIDATION | Accuracy {:.4f} | F1 {:.4f} | Precision {:.4f} | "
             "Recall {:.4f}".format(acc, f1, prec, rec))

    if f1 > best_score:
        save_config = {
                    'name': name,
                    'save_dir': ckpts_dir,
                    'global_step': global_step,
                    'clf_report': report
                }
        save_model(model=model, config=save_config)
        best_score = f1
    log.info("Validation end")

    model.train()
    return best_score



In [0]:

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)



In [0]:

if gpu and not torch.cuda.is_available():
    raise ValueError("GPU not supported or enabled on this system.")
use_gpu = gpu


In [0]:

log.info("Loading train dataset")
train_dataset = COVIDxFolder(train_imgs, train_labels, train_transforms(width,height))
train_loader = DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=True,
                            num_workers=n_threads,
                            pin_memory=use_gpu)
log.info("Number of training examples {}".format(len(train_dataset)))

log.info("Loading val dataset")
val_dataset = COVIDxFolder(val_imgs, val_labels, val_transforms(width, height))
val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=n_threads,
                        pin_memory=use_gpu)
log.info("Number of validation examples {}".format(len(val_dataset)))


In [0]:
# General
name = "COVIDNext50_FlowSIGMA"
gpu = True
batch_size = 64
n_threads = 20

# Model
# Model weights path
# weights = "./experiments/ckpts/<model.pth>"

# Optimizer
lr = 1e-4
weight_decay = 1e-3
lr_reduce_factor = 0.7
lr_reduce_patience = 5

ROOT = '/content/drive/My Drive/Colab Notebooks/COVID-Data/'
# Data
train_imgs = ROOT+"train"
train_labels = ROOT+"train_COVIDx.txt"

val_imgs = ROOT+"test"
val_labels = ROOT+"test_COVIDx.txt"

# Categories mapping
mapping = {
    'normal': 0,
    'pneumonia': 1,
    'COVID-19': 2
}
# Loss weigths order follows the order in the category mapping dict
loss_weights = [0.05, 0.05, 1.0]

width = 256
height = 256
n_classes = len(mapping)

# Training
epochs = 300
log_steps = 20
eval_steps = 40
ckpts_dir = ROOT+"ckpts"


In [35]:

state = None
'''
if weights:
    state = torch.load(weights)
    log.info("Loaded model weights from: {}".format(weights))
else:
    state = None
'''

state_dict = state["state_dict"] if state else None
model = COVIDNext50(n_classes=n_classes)
if state_dict:
    model = load_model_weights(model=model, state_dict=state_dict)

if use_gpu:
    model.cuda()
    model = torch.nn.DataParallel(model)
optim_layers = filter(lambda p: p.requires_grad, model.parameters())

# optimizer and lr scheduler
optimizer = Adam(optim_layers,
                    lr=lr,
                    weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer=optimizer,
                                factor=lr_reduce_factor,
                                patience=lr_reduce_patience,
                                mode='max',
                                min_lr=1e-7)

# Load the last global_step from the checkpoint if existing
global_step = 0 if state is None else state['global_step'] + 1

class_weights = to_device(torch.FloatTensor(loss_weights), gpu=use_gpu)
loss_fn = CrossEntropyLoss(reduction='mean', weight=class_weights)

# Reset the best metric score
best_score = -1
for epoch in range(epochs):
    log.info("Started epoch {}/{}".format(epoch + 1, epochs))
    for data in train_loader:
        imgs, labels = data
        imgs = to_device(imgs, gpu=use_gpu)
        labels = to_device(labels, gpu=use_gpu)

        logits = model(imgs)
        loss = loss_fn(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if global_step % log_steps == 0 and global_step > 0:
            #probs = probability(logits)
            probs = nn.functional.softmax(logits, dim=-1)
            preds = torch.argmax(probs, dim=1).detach().cpu().numpy()
            labels = labels.cpu().detach().numpy()
            acc, f1, _, _ = clf_metrics(preds, labels)
            lr = get_learning_rate(optimizer)

            log.info("Step {} | TRAINING batch: Loss {:.4f} | F1 {:.4f} | "
                        "Accuracy {:.4f} | LR {:.2e}".format(global_step,
                                                            loss.item(),
                                                            f1, acc,
                                                            lr))

        if global_step % eval_steps == 0 and global_step > 0:
            best_score = validate(val_loader,
                                    model,
                                    best_score=best_score,
                                    global_step=global_step
                                  )
            scheduler.step(best_score)
        global_step += 1



INFO:__main__:Started epoch 1/300
INFO:__main__:Step 20 | TRAINING batch: Loss 0.4689 | F1 0.8719 | Accuracy 0.8750 | LR 1.00e-04
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
INFO:__main__:Step 40 | TRAINING batch: Loss 0.4239 | F1 0.5981 | Accuracy 0.8906 | LR 1.00e-04
INFO:__main__:Validation started...
INFO:__main__:VALIDATION | Accuracy 0.7857 | F1 0.6316 | Precision 0.6457 | Recall 0.6700


FileNotFoundError: ignored